In [ ]:
from bs4 import BeautifulSoup
import urllib
from functools import partial
import mysql.connector
import re

In [ ]:
###Functions

def getPageSoup(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    request=urllib.request.Request(url,None,headers) 
    response = urllib.request.urlopen(request)
    soup = BeautifulSoup(response, 'html.parser')
    return soup

def getPageText(soup):
    page_text = soup.find_all('p')
    page_string = ""
    for p in page_text:
        page_string += p.get_text()
    return page_string

def getPageTitle(soup):
    return soup.title.string

def getAllLinks(soup):
    links =[]
    all_links = soup.find_all('a')
    for link in all_links:
        links.append(link.get('href'))
    return links
    

In [ ]:
original_url = "https://georgerrmartin.com/notablog/"
soup = getPageSoup(original_url)

In [ ]:
page_title = getPageTitle(soup)
page_text = getPageText(soup)
page_links = getAllLinks(soup)


In [ ]:
### Database connection
from functools import partial
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='', #sua senha
    database='nlp', #Nome database que quer acessar
)


def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)


db = partial(run_db_query, connection)

In [ ]:
### Database creation
# db('''CREATE TABLE websites(
# website_id INT NOT NULL AUTO_INCREMENT,
# url VARCHAR(150),
# website_content TEXT, 
# PRIMARY KEY (website_id)
# )
# ''')

In [ ]:
db(f"INSERT INTO websites(url, website_content) VALUES('{original_url}', '{page_text}')")

In [ ]:
for link in page_links:
    try:
        soup = getPageSoup(link)
        page_text = getPageText(soup)
        print(page_text)
        db(f"INSERT INTO websites(url, website_content) VALUES('{link}', '{page_text}')")
    except:
        print("An exception occurred")
    

In [ ]:
db('''commit''')

In [ ]:
connection.close()